In [1]:
# Loading Libraries 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras.backend import clear_session
from keras.layers import Dropout
#from keras.preprocessing.sequence import pad_sequences
from keras import layers
from keras.layers import LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import string

In [2]:
data = pd.read_excel("ArabicReviews.xlsx")
print(len(data))

704


In [3]:
data

,Sno,Reviews,Class,Company
0,0,سيئ جدا بعد الإصدار الجديد,-1,alahli_bank
1,1,ابلكيشن زباله بجد,-1,alahli_bank
2,2,سيئ التطبيق لايعمل,-1,alahli_bank
3,3,للأسف التطبيق للأسوأ كان جدا رائع وسهل وبسيط ا...,-1,alahli_bank
4,4,التحديث بطيئ جدا جدا عند الفتح,-1,alahli_bank
...,...,...,...,...
699,5429,ممكن اعرف متى راح يوصل الطلب,1,talbat
700,5430,رئع جدا بس مفيش خصم,1,talbat
701,5434,ممتااااز,1,talbat
702,5436,واوووو,1,talbat


In [22]:
NumpyData = data.head(300).to_numpy()
NumpyData = data.to_numpy()

In [23]:
NumpyData

array([[0, 'سيئ جدا بعد الإصدار الجديد', -1, 'alahli_bank'],
       [1, 'ابلكيشن زباله بجد', -1, 'alahli_bank'],
       [2, 'سيئ التطبيق لايعمل', -1, 'alahli_bank'],
       ...,
       [5434, 'ممتااااز', 1, 'talbat'],
       [5436, 'واوووو', 1, 'talbat'],
       [5439, 'عالمي بس عايز خصوم اكترررر', 1, 'talbat']], dtype=object)

In [49]:
for seq in range(0,10):
    print(seq)

0
1
2
3
4
5
6
7
8
9


In [50]:
from googletrans import Translator, constants
from pprint import pprint
translator=Translator()
filtered=[]
i=0
print(NumpyData[i][1])
print(NumpyData[i][2])


سيئ جدا بعد الإصدار الجديد
-1


In [70]:
for i in range(0,703):
    translation = translator.translate(NumpyData[i][1], src="ar")
    print({translation.text})
    print(NumpyData[i][1])
    print(NumpyData[i][2])
    filtered.append([NumpyData[i][1],translation.text,NumpyData[i][2]])
    print(str(i)+ "=====")
    i=i+1

In [71]:
len(filtered)

704

In [72]:
filtered

[['سيئ جدا بعد الإصدار الجديد', 'Too bad after the new version', -1],
 ['ابلكيشن زباله بجد', 'Really rubbish application', -1],
 ['سيئ التطبيق لايعمل', 'Bad app not working', -1],
 ['للأسف التطبيق للأسوأ كان جدا رائع وسهل وبسيط الآن معقد ولا يفتح معاك بسهولة ولا تجد كل الخدمات متاحة لك',
  'Unfortunately, for the worst, the application was very wonderful, easy, and simple. Now it is complicated and does not open with you easily, and you do not find all the services available to you',
  -1],
 ['التحديث بطيئ جدا جدا عند الفتح',
  'The update is very slow when opening',
  -1],
 ['مرررره زق من بعد اخر تحديث ما فتحلي ابدآ ويرفض اي كلمه مرور اكتبها او رقم البطاقه ..الاهلي خربتوا تطبيق اغلب الناس كانت معتمده عليه للأسف',
  'Pass it by after the last update, it never opened, and it refuses any password you write or the card number .. Al-Ahly ruined the application that most people used to rely on, unfortunately',
  -1],
 ['التطبيق ما يشتغل كل ما اعمل تسجيل دخول على حسابي ما يفتح شوفو لكم حل ان

In [74]:
for f in filtered:
    print(f)

['سيئ جدا بعد الإصدار الجديد', 'Too bad after the new version', -1]
['ابلكيشن زباله بجد', 'Really rubbish application', -1]
['سيئ التطبيق لايعمل', 'Bad app not working', -1]
['للأسف التطبيق للأسوأ كان جدا رائع وسهل وبسيط الآن معقد ولا يفتح معاك بسهولة ولا تجد كل الخدمات متاحة لك', 'Unfortunately, for the worst, the application was very wonderful, easy, and simple. Now it is complicated and does not open with you easily, and you do not find all the services available to you', -1]
['التحديث بطيئ جدا جدا عند الفتح', 'The update is very slow when opening', -1]
['مرررره زق من بعد اخر تحديث ما فتحلي ابدآ ويرفض اي كلمه مرور اكتبها او رقم البطاقه ..الاهلي خربتوا تطبيق اغلب الناس كانت معتمده عليه للأسف', 'Pass it by after the last update, it never opened, and it refuses any password you write or the card number .. Al-Ahly ruined the application that most people used to rely on, unfortunately', -1]
['التطبيق ما يشتغل كل ما اعمل تسجيل دخول على حسابي ما يفتح شوفو لكم حل انا معطله', 'The applicatio

In [77]:
df = pd.DataFrame(filtered, columns = ['Arabic','English','Class'])


In [78]:
df

,Arabic,English,Class
0,سيئ جدا بعد الإصدار الجديد,Too bad after the new version,-1
1,ابلكيشن زباله بجد,Really rubbish application,-1
2,سيئ التطبيق لايعمل,Bad app not working,-1
3,للأسف التطبيق للأسوأ كان جدا رائع وسهل وبسيط ا...,"Unfortunately, for the worst, the application ...",-1
4,التحديث بطيئ جدا جدا عند الفتح,The update is very slow when opening,-1
...,...,...,...
699,رائع وبالتوفيق يارب⚘,"Wonderful, and good luck, Lord ⚘",1
700,ممكن اعرف متى راح يوصل الطلب,May I know when the order will be delivered?,1
701,رئع جدا بس مفيش خصم,"Very cool, but there is no discount",1
702,ممتااااز,Excellent,1


In [79]:
df.to_excel("AabicToEnglishDataset.xlsx")  